I. Import all the python packages needed:

In [2]:
import csv
from pprint import pprint as pp
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import re
import country_converter as coco
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

III. Combine Component A,B,C 

In [102]:
# combine artist_detail file with spotify_api file
df_spotify = pd.read_csv('track_artist_audio_features_final.csv', encoding='utf-8')

df_artist = pd.read_csv('artists_details_final.csv', encoding='utf-8')
df_artist = df_artist.rename(columns={'spotify track artist audio.artist_id': 'artist_id'}).dropna(subset=['artist_id'])
df_artist = df_artist[['artist_id', 'birth_year', 'artist_country', 'gender', 'band']]

combined = pd.merge(df_spotify, df_artist, how='left', on='artist_id')

combined.to_csv('spotify_combined_w_artist_details.csv', index=False)


In [290]:
# combine all the file
df_spotify_w_artist = pd.read_csv('spotify_combined_w_artist_details.csv', encoding='utf-8')

df_lyrics = pd.read_csv('lyrics_final.csv', encoding='utf-8')
df_lyrics = df_lyrics.drop(columns=['song', 'artist_name', 'artist_id', 'lyrics', 'repetitions_rate_median'])

combined = pd.merge(df_spotify_w_artist, df_lyrics, how='left', on='track_id')
combined.to_csv('final.csv', index=False)


IV: Data Analysis

Step1: Know more about our data

In [329]:
df = pd.read_csv('final.csv', encoding='utf-8')
df.describe(include='all')

,song,track_id,release_date,song_popularity,duration(ms),explicit,artist_name,artist_id,artist_follower_no,artist_genres,artist_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,birth_year,artist_country,gender,band,repetitions_rate,mean_sentence_len,mean_word_len,unique_words_rate,total_words,total_char,count_word_i,count_word_you,count_word_oh,count_word_know,count_word_it,count_word_like,count_word_got,count_word_and,count_word_yeah,count_word_we,count_word_one,count_word_love,count_word_the,count_word_go,count_word_time,count_word_get,count_word_me,count_word_when,count_word_come,count_word_all
count,7201,7201,7201,7201.000000,7.201000e+03,7201,7201,7201,7.201000e+03,6623,7201.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,7153.000000,2416.000000,5135,1759,5464,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000
unique,6785,7201,2476,NaN,NaN,2,3071,3071,NaN,2173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Home,4eOycahNij77maJicwgHvy,2013-01-01,NaN,NaN,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02,NaN,['pop'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,male,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7,1,102,NaN,NaN,6682,103,103,NaN,132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3041,1083,3103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,51.909874,2.836999e+05,NaN,NaN,NaN,5.315898e+06,NaN,57.343147,0.237389,0.556860,0.653853,0.099938,5.253600,0.186045,-7.661730,0.664756,0.064116,121.141731,3.950370,0.471709,1973.685844,NaN,NaN,NaN,38.505920,31.739457,5.063576,0.221031,311.104022,1115.302589,15.612344,12.412159,2.301433,1.889736,5.855756,1.980120,1.172908,6.188858,1.408692,3.058715,0.993528,1.868932,10.042996,1.139390,0.965326,1.130143,5.037448,1.439667,0.855294,2.283634
std,NaN,NaN,NaN,18.108126,4.465146e+05,NaN,NaN,NaN,1.217781e+07,NaN,21.192055,0.291704,0.160001,0.226139,0.243613,3.572906,0.155160,4.220787,0.472109,0.067928,28.276143,0.259436,0.247863,23.910258,NaN,NaN,NaN,13.995913,11.155487,0.383404,0.085498,160.170650,583.789186,14.423815,12.278089,7.757792,4.230099,7.907178,6.704641,2.896945,6.248694,5.188592,5.902308,2.597100,5.670952,9.148309,2.983304,2.470861,3.261466,7.197976,2.594247,2.908763,3.603434
min,NaN,NaN,NaN,1.000000,5.191900e+04,NaN,NaN,NaN,0.000000e+00,NaN,0.000000,0.000001,0.058500,0.000020,0.000000,0.000000,0.015700,-44.907000,0.000000,0.022700,43.509000,3.000000,0.000010,1734.000000,NaN,NaN,NaN,2.400000,10.500000,3.000000,0.010000,5.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,41.000000,1.979730e+05,NaN,NaN,NaN,1.249330e+05,NaN,48.000000,0.012600,0.457000,0.505000,0.000000,2.000000,0.094800,-9.237000,0.000000,0.033200,99.995000,4.000000,0.273000,1964.000000,NaN,NaN,NaN,28.970000,26.192500,4.830000,0.160000,210.000000,758.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,54.000000,2.258530e+05,NaN,NaN,NaN,9.554630e+05,NaN,62.000000,0.094200,0.565000,0.696000,0.000053,5.000000,0.122000,-6.557000,1.000000,0.042200,119.979000,4.000000,0.462000,1981.000000,NaN,NaN

Step2: Modify values in each column to the same form to give consistency

Step2a: Change release_date accuracy to 3 significant figures (e.g. 1980s)

In [330]:
for row in df.index:
    release_date = df.loc[row, 'release_date']
    if str(release_date) != 'nan':
        df.loc[row, 'release_date'] = release_date[:3]+'0s'
        
# df.describe(include='all')

(Step2b): Change artist_genres into bigger categories. This could be done while working on spotify API. 

In [331]:
df = df.sort_values(by='artist_genres')

for row in df.index:
    genres = df.loc[row, 'artist_genres']
    if str(genres) == 'nan':
        break
    else:
        if str(genres).find('worship') != -1 or str(genres).find('christian') != -1 or str(genres).find('ccm') != -1:
            df.loc[row, 'artist_genres'] = 'christian'
        elif str(genres).find('electro') != -1 or str(genres).find('8-bit') != -1 or str(genres).find('digit') != -1 or str(genres).find('chill') != -1 or str(genres).find('wave') != -1 or str(genres).find('ambient') != -1 or str(genres).find('core') != -1:
            df.loc[row, 'artist_genres'] = 'electronic'
        elif str(genres).find('dance') != -1 or str(genres).find('disco') != -1 or str(genres).find('house') != -1 or str(genres).find('edm') != -1:
            df.loc[row, 'artist_genres'] = 'electronic'  
        elif str(genres).find('metal') != -1 or str(genres).find('rock') != -1 or str(genres).find('psych') != -1 or str(genres).find('punk') != -1:
            df.loc[row, 'artist_genres'] = 'metal / rock'
        elif str(genres).find('hip') != -1 or str(genres).find('rap') != -1:
            df.loc[row, 'artist_genres'] = 'hip hop / rap'    
        elif str(genres).find('environment') != -1 or str(genres).find('sleep') != -1 or str(genres).find('water') != -1 or str(genres).find('zen') != -1:
            df.loc[row, 'artist_genres'] = 'sleep / environmental'     
        elif str(genres).find('cappella') != -1:
            df.loc[row, 'artist_genres'] = 'acappella'   
        elif str(genres).find('country') != -1 or str(genres).find('folk') != -1:
            df.loc[row, 'artist_genres'] = 'country / folk'
        elif str(genres).find('pop') != -1:
            df.loc[row, 'artist_genres'] = 'pop'
        elif str(genres).find('soul') != -1 or str(genres).find('r&b') != -1 or str(genres).find('blue') != -1 or str(genres).find('jazz') != -1:
            df.loc[row, 'artist_genres'] = 'r&b / soul / jazz'
        elif str(genres).find('orchestra') != -1 or str(genres).find('\'classical\'') != -1:
            df.loc[row, 'artist_genres'] = 'orchestra / classical'    
        elif str(genres).find('band') != -1 or str(genres).find('core') != -1:
            df.loc[row, 'artist_genres'] = 'band' 
        elif str(genres).find('vgm') != -1 or str(genres).find('soundtrack') != -1 or str(genres).find('disney') != -1 or str(genres).find('hollywood') != -1:
            df.loc[row, 'artist_genres'] = 'soundtrack'  
        elif str(genres).find('easy') != -1 or str(genres).find('lounge') != -1:
            df.loc[row, 'artist_genres'] = 'lounge' 
        elif str(genres).find('karaoke') != -1:
            df.loc[row, 'artist_genres'] = 'karaoke' 
        else: 
            df.loc[row, 'artist_genres'] = 'N/A'

df = df.sort_values(by='artist_genres', ascending=True)
                    
# df.describe(include='all')

(Step2c): Remove audio feature for sleep / environmental music. This could be done while working on spotify API.

In [332]:
for row in df.index:
    if df.loc[row, 'artist_genres'] == 'sleep / environmental':
        df.loc[row, 'time_signature'] = ''
        df.loc[row, 'acousticness'] = ''
        df.loc[row, 'danceability'] = ''
        df.loc[row, 'energy'] = ''
        df.loc[row, 'instrumentalness'] = ''
        df.loc[row, 'key'] = ''
        df.loc[row, 'liveness'] = ''
        df.loc[row, 'loudness'] = ''
        df.loc[row, 'mode'] = ''
        df.loc[row, 'speechiness'] = ''
        df.loc[row, 'tempo'] = ''
        df.loc[row, 'time_signature'] = ''
        df.loc[row, 'valence'] = ''

(Step2d): Remove birth year for bands

In [333]:
for row in df.index: 
    if df.loc[row, 'band'] == True:
        df.loc[row, 'birth_year'] = 'nan'

Step2e: Change release_date accuracy to 3 significant figures (e.g. 1980s)

In [334]:
for row in df.index:
    birth_year = str(df.loc[row, 'birth_year'])
    if str(birth_year) != 'nan':
        df.loc[row, 'birth_year'] = birth_year[:3]+'0s'
        
# df.describe(include='all')


,song,track_id,release_date,song_popularity,duration(ms),explicit,artist_name,artist_id,artist_follower_no,artist_genres,artist_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,birth_year,artist_country,gender,band,repetitions_rate,mean_sentence_len,mean_word_len,unique_words_rate,total_words,total_char,count_word_i,count_word_you,count_word_oh,count_word_know,count_word_it,count_word_like,count_word_got,count_word_and,count_word_yeah,count_word_we,count_word_one,count_word_love,count_word_the,count_word_go,count_word_time,count_word_get,count_word_me,count_word_when,count_word_come,count_word_all
count,7201,7201,7201,7201.000000,7.201000e+03,7201,7201,7201,7.201000e+03,6623,7201.000000,7166,7166,7166,7166.0,7166.0,7166.000,7166,7166.0,7166,7166,7166.0,7166,4755,5135,1759,5464,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000
unique,6785,7201,10,NaN,NaN,2,3071,3071,NaN,15,NaN,2717,805,979,2844.0,13.0,1225.000,5157,3.0,996,6380,4.0,1172,20,55,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Home,268fHX2HCDAbB4rq7BS2XT,2010s,NaN,NaN,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02,NaN,metal / rock,NaN,,,,0.0,0.0,0.106,,1.0,,,4.0,,nan,United States,male,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,7,1,4016,NaN,NaN,6682,103,103,NaN,2189,NaN,78,78,78,2285.0,868.0,87.000,78,4709.0,78,78,6622.0,78,2361,3041,1083,3103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,51.909874,2.836999e+05,NaN,NaN,NaN,5.315898e+06,NaN,57.343147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.505920,31.739457,5.063576,0.221031,311.104022,1115.302589,15.612344,12.412159,2.301433,1.889736,5.855756,1.980120,1.172908,6.188858,1.408692,3.058715,0.993528,1.868932,10.042996,1.139390,0.965326,1.130143,5.037448,1.439667,0.855294,2.283634
std,NaN,NaN,NaN,18.108126,4.465146e+05,NaN,NaN,NaN,1.217781e+07,NaN,21.192055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.995913,11.155487,0.383404,0.085498,160.170650,583.789186,14.423815,12.278089,7.757792,4.230099,7.907178,6.704641,2.896945,6.248694,5.188592,5.902308,2.597100,5.670952,9.148309,2.983304,2.470861,3.261466,7.197976,2.594247,2.908763,3.603434
min,NaN,NaN,NaN,1.000000,5.191900e+04,NaN,NaN,NaN,0.000000e+00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000,10.500000,3.000000,0.010000,5.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,41.000000,1.979730e+05,NaN,NaN,NaN,1.249330e+05,NaN,48.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.970000,26.192500,4.830000,0.160000,210.000000,758.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,54.000000,2.258530e+05,NaN,NaN,NaN,9.554630e+05,NaN,62.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.260000,30.820000,5.050000,0.210000,288.000000,1024.000000,12.000000,9.000000,0.000000,1.000000,3.000000,0.000000,0.000000,5.000000,0.000000,1.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000
75%,NaN,NaN,NaN,66.000000,2.639460e+05,NaN,NaN,NaN,4.009684e+06,NaN,72.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.557500,36.025000,5.280000,0.260000,376

Step2f: Drop unuseful columns for analysis

In [338]:
df = df.drop(columns=['song', 'track_id', 'artist_id'])

df.describe(include='all')

,release_date,song_popularity,duration(ms),explicit,artist_name,artist_follower_no,artist_genres,artist_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,birth_year,artist_country,gender,band,repetitions_rate,mean_sentence_len,mean_word_len,unique_words_rate,total_words,total_char,count_word_i,count_word_you,count_word_oh,count_word_know,count_word_it,count_word_like,count_word_got,count_word_and,count_word_yeah,count_word_we,count_word_one,count_word_love,count_word_the,count_word_go,count_word_time,count_word_get,count_word_me,count_word_when,count_word_come,count_word_all
count,7201,7201.000000,7.201000e+03,7201,7201,7.201000e+03,6378,7201.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,7088.000000,2394,5135,1759,5464,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000,4326.000000
unique,10,NaN,NaN,2,3071,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,55,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2010s,NaN,NaN,False,Taylor Swift,NaN,metal / rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1980s,United States,male,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4016,NaN,NaN,6682,103,NaN,2189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689,3041,1083,3103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,51.909874,2.836999e+05,NaN,NaN,5.315898e+06,NaN,57.343147,0.234114,0.560011,0.654176,0.093363,5.254233,0.183286,-7.519805,0.664362,0.062855,121.355241,3.951185,0.475572,NaN,NaN,NaN,NaN,38.505920,31.739457,5.063576,0.221031,311.104022,1115.302589,15.612344,12.412159,2.301433,1.889736,5.855756,1.980120,1.172908,6.188858,1.408692,3.058715,0.993528,1.868932,10.042996,1.139390,0.965326,1.130143,5.037448,1.439667,0.855294,2.283634
std,NaN,18.108126,4.465146e+05,NaN,NaN,1.217781e+07,NaN,21.192055,0.289355,0.156850,0.224133,0.233510,3.568908,0.149729,3.905956,0.472246,0.061866,28.189055,0.251736,0.245466,NaN,NaN,NaN,NaN,13.995913,11.155487,0.383404,0.085498,160.170650,583.789186,14.423815,12.278089,7.757792,4.230099,7.907178,6.704641,2.896945,6.248694,5.188592,5.902308,2.597100,5.670952,9.148309,2.983304,2.470861,3.261466,7.197976,2.594247,2.908763,3.603434
min,NaN,1.000000,5.191900e+04,NaN,NaN,0.000000e+00,NaN,0.000000,0.000001,0.058600,0.000088,0.000000,0.000000,0.015700,-43.185000,0.000000,0.022700,43.509000,3.000000,0.000010,NaN,NaN,NaN,NaN,2.400000,10.500000,3.000000,0.010000,5.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,41.000000,1.979730e+05,NaN,NaN,1.249330e+05,NaN,48.000000,0.012475,0.460000,0.507000,0.000000,2.000000,0.094600,-9.147250,0.000000,0.033200,100.010000,4.000000,0.278750,NaN,NaN,NaN,NaN,28.970000,26.192500,4.830000,0.160000,210.000000,758.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,NaN,54.000000,2.258530e+05,NaN,NaN,9.554630e+05,NaN,62.000000,0.091750,0.567500,0.696000,0.000048,5.000000,0.122000,-6.522000,1.000000,0.042100,119.988000,4.000000,0.466500,NaN,NaN,NaN,NaN,37.260000,30.820000,5.050000,0.210000,288.000000,1024.000000,12.000000,9.000000,0.000000,1.000000,3.000000,0.000000,0.000000,5.000000,0.000000,1.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000,3.00000

In [339]:
df.to_csv('final_consistent.csv', index=False)

Step3: Modification by looking closer to our data

In [193]:
df = pd.read_csv('final_consistent.csv', encoding='utf-8')

Classify columns into different types

In [194]:
# show all the columns
df.columns

Index(['release_date', 'song_popularity', 'duration(ms)', 'explicit',
       'artist_name', 'artist_follower_no', 'artist_genres',
       'artist_popularity', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'birth_year',
       'artist_country', 'gender', 'band', 'repetitions_rate',
       'mean_sentence_len', 'mean_word_len', 'unique_words_rate',
       'total_words', 'total_char', 'count_word_i', 'count_word_you',
       'count_word_oh', 'count_word_know', 'count_word_it', 'count_word_like',
       'count_word_got', 'count_word_and', 'count_word_yeah', 'count_word_we',
       'count_word_one', 'count_word_love', 'count_word_the', 'count_word_go',
       'count_word_time', 'count_word_get', 'count_word_me', 'count_word_when',
       'count_word_come', 'count_word_all'],
      dtype='object')

In [195]:
# will be used later after some data manipulation

# df_numerical = df[['song_popularity', 'duration(ms)', 'artist_follower_no', 'artist_popularity', 
#                    'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 
#                    'repetitions_rate', 'mean_sentence_len', 'mean_word_len', 'unique_words_rate',
#                    'total_words', 'total_char', 'count_word_i', 'count_word_you', 'count_word_oh', 'count_word_know', 'count_word_it', 'count_word_like', 'count_word_got', 'count_word_and', 'count_word_yeah', 'count_word_we',
#                    'count_word_one', 'count_word_love', 'count_word_the', 'count_word_go', 'count_word_time', 'count_word_get', 'count_word_me', 'count_word_when', 'count_word_come', 'count_word_all']]
# df_categories = df[['release_date', 'explicit', 'artist_genres', 'birth_year', 'artist_country', 'gender', 'band']]
# df_unique = df[['artist_name']]

Less than 100 data point is not adequate for a good statistical distribution. Hence, we remove groups with less than 100 data points or combine with other groups. This needs to be done for categories group only.

We write a function to remove smaller groups

In [196]:
def remove_small_groups(column_name):
    counts = df[column_name].value_counts()
    remove = list(counts[counts < 100].index)
    df[column_name] = df[column_name].replace(remove,np.nan)
    

In [197]:
# combine release_date into bigger groups
df['release_date'] = df['release_date'].replace(['1930s','1940s', '1950s', '1960s'],'1960s and earlier')

df['release_date'].value_counts()

2010s                4016
2000s                1693
1990s                 604
1980s                 311
1970s                 270
1960s and earlier     205
2020s                 102
Name: release_date, dtype: int64

In [198]:
# karaokes are usually not published by the artist on spotify, hence we remove artist information fetched via spotify
df = df.sort_values(by='artist_genres')

for row in df.index:
    genres = df.loc[row, 'artist_genres']
    if str(genres) == 'nan':
        break
    else:
        if genres == 'karaoke':
            df.loc[row, 'artist_follower_no'] = np.nan
            df.loc[row, 'artist_popularity'] = np.nan


# combine artist_genres into bigger groups / remove genres
remove_small_groups('artist_genres')

df['artist_genres'].value_counts()

metal / rock         2189
electronic           2165
pop                   765
country / folk        379
hip hop / rap         238
christian             225
r&b / soul / jazz     151
Name: artist_genres, dtype: int64

In [199]:
# combine birth_year into bigger groups / remove birth_year
df['birth_year'] = df['birth_year'].replace(['1990s', '2000s','2010s', '2020s'],'1990s and younger')

remove_small_groups('birth_year')

df['birth_year'].value_counts()

1980s                689
1990s and younger    547
1970s                398
1960s                236
1940s                200
1950s                153
Name: birth_year, dtype: int64

In [200]:
# combine artist_country into bigger groups / remove artist_country

df['artist_country'] = df['artist_country'].replace(['Italy', 'France'], 'Italy / France')

remove_small_groups('artist_country')

df['artist_country'].value_counts()

United States     3041
United Kingdom     982
Canada             193
Australia          171
Norway             163
Italy / France     142
Sweden             105
Name: artist_country, dtype: int64

In [201]:
df.to_csv('final_cleaned.csv', index = False)